<a href="https://colab.research.google.com/github/sm2hashmi/Syed-Muhammad-Masab-Hashmi/blob/master/Deep%20Learning%20Project%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy
import pandas
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline


Using TensorFlow backend.


In [0]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)


In [4]:
# load dataset
from google.colab import files
uploaded = files.upload()
import io
dataframe = pandas.read_csv(io.BytesIO(uploaded['iris.csv']))
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]


Saving iris.csv to iris.csv


In [0]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)


In [0]:
def create_model():
   model = Sequential()
   model.add(layers.Dense(8, activation= 'relu', input_shape = (4,)))
   model.add(layers.Dense(3, activation = 'softmax'))
    
   model.compile(optimizer='Adam', loss ='categorical_crossentropy', metrics = ['accuracy'])
  
   return model

In [13]:
estimator = KerasClassifier(build_fn=create_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))



Baseline: 97.33% (5.33%)


In [23]:
numpy.random.seed(seed)
estimators = []

estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Standardized: 95.95% (4.45%)


In [27]:
def create_large():
   model = Sequential()
   model.add(layers.Dense(16, activation= 'relu', input_shape = (4,)))
   model.add(layers.Dense(3, activation = 'softmax'))
    
   model.compile(optimizer='Adam', loss ='categorical_crossentropy', metrics = ['accuracy'])
  
   return model
  
estimators = []
estimators.append(('mlp', KerasClassifier(build_fn=create_large, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 96.62% (4.51%)


**Using Functional API**

In [0]:
from keras.layers import Input
from keras.models import Model
  
visible = Input(shape = (4,))
hidden1 = layers.Dense(8, activation = 'relu')(visible)
output = layers.Dense(3, activation='softmax')(hidden1)
model = Model(inputs=visible,outputs=output)
model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'],loss_weights=None, sample_weight_mode=None, weighted_metrics=None, )
